# Validation through experimentation

This notebook implements several experiments that aim to validate the behavior of the `fdsim` package. Every experiment consists of two or more scenarios that are compared in terms of different performance metrics. The expected effect of making changes in the simulation environment (i.e., changing the _scenario_) is known in advance: it is either obtained by simple logic or from domain experts.

In this notebook, we consider two types of experiments: 
* _A/B tests_: situations in which we consider exactly two alternative scenarios.
* _Multi-scenario experiments_: situations in which we consider three or more alternative scenarios.

The experiments that we will perform using these two types of experiments are the following.

1. Changing vehicles.
    - a) Remove TS from Hendrik (big  effect)
    - b) Remove TS from Ysbrand (small effect)
    - c) Remove second TS from Amstelveen (almost no effect)
    - d) Add RV to Zebra (small effect)
    - e) Remove HV Diemen
    - f) Add WO Nico (small positive effect)
2. Moving stations
    - a) Moving Osdorp up to the N200
    - b) Movements suggested in paper
3. Adding and closing stations
    - a) Replicating closing of Landelijk Noord (minimal effect)
    - b) Replicating closing of Ouderkerk aan de Amstel (minimal effect)
    - c) Add station in Bovenkerk
4. Station status cycles
    - a) Closing Victor at night (23h - 7h, 7 days a week)
    - b) Diemen part time outside office hours
5. Back-up protocol
    - a) Add backup system for Amstelveen
6. Changing forecast
    - a) Stress test the system by increasing incident rates with different factors

## 0. Preparations
Before we start experimenting, we import the necessary modules, load the required data and initialize a simulator that will represent our baseline for all experiments. By using the same baseline, we can better compare the influence of different decisions/changes.

In [ ]:
# imports
import sys
sys.path.append("..")
import copy
import numpy as np
import pandas as pd
from fdsim.experiments import ABTest, MultiScenarioExperiment
from fdsim.simulation import Simulator
from fdsim.evaluation import Evaluator
from fdsim.helpers import quick_load_simulator

# data
deployments = pd.read_csv("../../Data/inzetten_2008-heden.csv", sep=";", decimal=",", low_memory=False, dtype={"hub_incident_id": int})
incidents = pd.read_csv("../../Data/incidenten_2008-heden.csv", sep=";", decimal=",", low_memory=False, dtype={"dim_incident_id": int})
station_locations = pd.read_excel("../../Data/kazernepositie en voertuigen.xlsx", sep=";", decimal=".")
resource_allocation = pd.read_csv("../../Data/resource_allocation.csv", sep=";", decimal=".")

# baseline situation:
resource_allocation

In [ ]:
# obtain service areas
def get_service_areas(data, stations, vehicle_col="voertuig_groep", station_col="kazerne_groep", loc_col="hub_vak_bk", id_col="hub_incident_id"):
    counts = data.groupby([loc_col, station_col])[id_col].count().reset_index()
    counts[station_col] = counts[station_col].str.upper()
    counts = counts[np.in1d(counts[station_col], stations)]
    maxes = pd.Series(counts.groupby(loc_col).apply(lambda x: x[station_col].loc[x[id_col].argmax()]), name=station_col).reset_index()

    sa = maxes.groupby(station_col)[loc_col].apply(lambda x: np.unique(x))
    return sa.T.to_dict()

merged = deployments.merge(
    incidents[["dim_incident_id", "hub_vak_bk"]].dropna().astype(int),
    left_on="hub_incident_id",
    right_on="dim_incident_id",
    how="inner"
)

merged = merged[merged["hub_vak_bk"].astype(str).str[0:2] == "13"]

service_areas = get_service_areas(merged, station_locations["kazerne"])
service_areas["LANDELIJK NOORD"]

In [ ]:
# initialization
base_simulator = Simulator(incidents, deployments, station_locations, resource_allocation, predictor="basic",
                           load_response_data=True, data_dir="../data")
base_simulator.save_simulator_object("baseline_simulator.pickle")

# 1. Changing vehicles

## Experiment 1.a: Remove TS from station Hendrik.
Expected effect: large effect on overall response time, since it is a busy station.

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has no TS at station Hendrik
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.set_vehicles("HENDRIK", "TS", 0)

evaluator = Evaluator()
# we add two metric on the response time and on-time rate of TS vehicles and
# one on the other vehicle types to verify if there is no effect
evaluator.add_metric("response_time", name="TS response time", vehicles="TS",
                     description="Response time of all TS deployments.")
evaluator.add_metric("delay", name="Service area delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["HENDRIK"],
                     description="Response time delay of first TS for prio 1 incidents in affected service area.")
evaluator.add_metric("on_time", name="First TS on-time", vehicles="TS", first_only=True, prios=1,
                     description="On-time rate of first TS.", quantiles=None)
evaluator.add_metric("response_time", name="Non-TS", vehicles=["RV", "HV", "WO"],
                     description="Response time of unaffected vehicle types.")


experiment1a = ABTest(evaluator=evaluator)
experiment1a.set_scenario_A(sim_a, description="Baseline situation")
experiment1a.set_scenario_B(sim_b, description="No TS at station Hendrik")
experiment1a.run()

In [ ]:
experiment1a.print_results()

## Experiment 1.b: Remove TS from station Ijsbrand.
Expected effect: small effect on overall response time, since it is a relatively calm station.

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has no TS at station Ijsbrand
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.set_vehicles("IJSBRAND", "TS", 0)

evaluator = Evaluator()
evaluator.add_metric("response_time", name="TS response time", vehicles="TS",
                     description="Response time of all TS deployments.")
evaluator.add_metric("delay", name="Service area delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["IJSBRAND"],
                     description="Response time delay of first TS for prio 1 incidents in affected service area.")
evaluator.add_metric("on_time", name="First TS on-time", vehicles="TS", first_only=True, prios=1,
                     description="On-time rate of first TS.", quantiles=None)
evaluator.add_metric("response_time", name="Non-TS", vehicles=["RV", "HV", "WO"],
                     description="Response time of unaffected vehicle types.")

# Let's use the same evaluator as in 1a.
experiment1b = ABTest(evaluator=evaluator)
experiment1b.set_scenario_A(sim_a, description="Baseline situation")
experiment1b.set_scenario_B(sim_b, description="No TS at station Ijsbrand")
experiment1b.run()
experiment1b.print_results()

## Experiment 1.c: Remove second TS from station Amstelveen.
Expected effect: almost no effect on overall response time, since it is the second vehicle.

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has only one TS at station Amstelveen
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.set_vehicles("AMSTELVEEN", "TS", 1)

evaluator = Evaluator()
evaluator.add_metric("response_time", name="TS response time", vehicles="TS",
                     description="Response time of all TS deployments.")
evaluator.add_metric("delay", name="Service area delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["AMSTELVEEN"],
                     description="Response time delay of first TS for prio 1 incidents in affected service area.")
evaluator.add_metric("on_time", name="First TS on-time", vehicles="TS", first_only=True, prios=1,
                     description="On-time rate of first TS.", quantiles=None)
evaluator.add_metric("response_time", name="Non-TS", vehicles=["RV", "HV", "WO"],
                     description="Response time of unaffected vehicle types.")

# Let's use the same evaluator as in 1a and 1b.
experiment1c = ABTest(evaluator=evaluator)
experiment1c.set_scenario_A(sim_a, description="Baseline situation")
experiment1c.set_scenario_B(sim_b, description="Only one TS at station Amstelveen")
experiment1c.run()
experiment1c.print_results()

## Experiment 1.d: Add RV to station Zebra.
Expected effect: small positive effect.

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has only one TS at station Amstelveen
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.set_vehicles("ZEBRA", "RV", 1)
sim_b.set_crews("ZEBRA", "RV", 1)

evaluator = Evaluator()

evaluator = Evaluator()
evaluator.add_metric("response_time", name="TS response time", vehicles="TS",
                     description="Response time of all TS deployments.")

evaluator.add_metric("response_time", name="RV response time", vehicles="RV",
                     description="Response time of all RV deployments.")
evaluator.add_metric("on_time", name="First RV on-time", vehicles="RV", first_only=True,
                     description="On-time rate of first RV.", quantiles=None)
evaluator.add_metric("delay", name="Service area delay", vehicles="RV", first_only=True, prios=1, locations=service_areas["ZEBRA"],
                     description="Response time delay of first RV for prio 1 incidents in affected service area.")
evaluator.add_metric("response_time", name="Non-RV", vehicles=["TS", "HV", "WO"],
                     description="Response time of unaffected vehicle types.")

experiment1d = ABTest(evaluator=evaluator)
experiment1d.set_scenario_A(sim_a, description="Baseline situation")
experiment1d.set_scenario_B(sim_b, description="Additional RV positioned at station Zebra.")
experiment1d.run()
experiment1d.print_results()

## Experiment 1.e: Remove HV from station Diemen.
Expected effect: negative effect.

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has only one TS at station Amstelveen
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.set_vehicles("DIEMEN", "HV", 0)

evaluator = Evaluator()
# we add two metric on the response time and on-time rate of TS vehicles and
# one on the other vehicle types to verify if there is no effect
evaluator.add_metric("response_time", name="HV response time", vehicles="HV",
                     description="Response time of all HV deployments.")
evaluator.add_metric("on_time", name="First HV on-time", vehicles="HV", first_only=True,
                     description="On-time rate of first HV.", quantiles=None)
evaluator.add_metric("delay", name="Service area delay", vehicles="HV", first_only=True, prios=1, locations=service_areas["DIEMEN"],
                     description="Response time delay of first RV for prio 1 incidents in affected service area.")
evaluator.add_metric("response_time", name="Non-HV", vehicles=["TS", "RV", "WO"],
                     description="Response time of unaffected vehicle types.")

experiment1e = ABTest(evaluator=evaluator)
experiment1e.set_scenario_A(sim_a, description="Baseline situation")
experiment1e.set_scenario_B(sim_b, description="Removed HV from station Diemen.")
experiment1e.run()
experiment1e.print_results()

## Experiment 1.f: Add WO to station Nico.
Expected effect: small positive effect.

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has only one TS at station Amstelveen
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.set_vehicles("NICO", "WO", 1)
sim_b.set_crews("NICO", "WO", 1)

evaluator = Evaluator()
# we add two metric on the response time and on-time rate of TS vehicles and
# one on the other vehicle types to verify if there is no effect
evaluator.add_metric("response_time", name="WO response time", vehicles="WO",
                     description="Response time of all WO deployments.")
evaluator.add_metric("on_time", name="First WO on-time", vehicles="WO", first_only=True,
                     description="On-time rate of first WO.", quantiles=None)
evaluator.add_metric("delay", name="Service area delay", vehicles="WO", first_only=True, prios=1, locations=service_areas["NICO"],
                     description="Response time delay of first WO for prio 1 incidents in affected service area.")
evaluator.add_metric("response_time", name="Non-WO", vehicles=["TS", "RV", "HV"],
                     description="Response time of unaffected vehicle types.")

experiment1f = ABTest(evaluator=evaluator)
experiment1f.set_scenario_A(sim_a, description="Baseline situation")
experiment1f.set_scenario_B(sim_b, description="Removed HV from station Diemen.")
experiment1f.run()
experiment1f.print_results()

# 2. Moving stations

We have couple of experiments where we move the locations of stations.

## Experiment 2.a: Moving Osdorp to the N200.
We move station Osdorp up towards the N200 and closer to the industrial site above the N200. Specifically, we place it in the demand location with id `13781452`.

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has relocated station Osdorp
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.move_station("OSDORP", "13781452")

evaluator = Evaluator()
evaluator.add_metric("response_time", name="response time", description="Overall response time of all deployments.")
evaluator.add_metric("response_time", name="response time Osdorp", locations=service_areas["OSDORP"],
                     description="Overall response time of all deployments in Osdorp's original service area.")
evaluator.add_metric("delay", name="Service area delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["ZEBRA"],
                     description="Response time delay of first TS for prio 1 incidents in affected service area.")
evaluator.add_metric("on_time", name="First TS on-time", vehicles="TS", first_only=True,
                     description="On-time rate of first TS.", quantiles=None)

experiment1f = ABTest(evaluator=evaluator)
experiment1f.set_scenario_A(sim_a, description="Baseline situation")
experiment1f.set_scenario_B(sim_b, description="Station Osdorp moved up to the N200.")
experiment1f.run()

In [ ]:
experiment1f.print_results()

## Experiment 2.b: Testing suggested station positions of previous study.

A paper (...) suggested three station relocations that should lead to improved coverage. Let's see if our simulator would agree.

In [ ]:
# this is best tested using the MultiFactorExperiment, which is not ready for use yet..

# 3. Adding and removing stations
In this section, we replicate two closings of stations that happened in the past years: stations "Landelijk Noord" and "Ouderkerk aan de Amstel" were closed, with an apparently minimal effect. The simulator should give similar results.

## Experiment 3.a: Closing of Landelijk Noord

In [ ]:
# create resource allocation with Landelijk Noord
# having 1 TS with a part time crew
resources_with_landelijknoord = resource_allocation.copy()
new_resources = [1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
resources_with_landelijknoord.set_index("kazerne", inplace=True)
resources_with_landelijknoord.loc["Landelijk Noord", :] = new_resources
resources_with_landelijknoord = resources_with_landelijknoord.astype(int).reset_index()

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has the original station Landelijk Noord
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.set_resource_allocation(resources_with_landelijknoord)

In [ ]:
evaluator = Evaluator()
evaluator.add_metric("response_time", name="response time", description="Overall response time of all deployments.")
evaluator.add_metric("on_time", name="First TS on-time", vehicles="TS", first_only=True,
                     description="Overall on-time rate of first TS.", quantiles=None)
evaluator.add_metric("delay", name="Service area delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["LANDELIJK NOORD"],
                     description="Response time delay of first TS for prio 1 incidents in affected service area.")

In [ ]:
# Let's use the same evaluator as in 1a and 1b.
experiment3a = ABTest(evaluator=evaluator)
experiment3a.set_scenario_A(sim_a, description="Landelijk Noord closed.")
experiment3a.set_scenario_B(sim_b, description="Landelijk Noord operating (1 TS with part time crew).")
experiment3a.run()
experiment3a.print_results()

## Experiment 3.b: Closing of Ouderkerk aan de Amstel

In [ ]:
# create resource allocation with Ouderkerk aan de Amstel
# having 1 TS with a part time crew
resources_with_ouderkerk = resource_allocation.copy()
new_resources = [1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
resources_with_ouderkerk.set_index("kazerne", inplace=True)
resources_with_ouderkerk.loc["Ouderkerk aan de Amstel", :] = new_resources
resources_with_ouderkerk = resources_with_ouderkerk.astype(int).reset_index()

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has the original station Ouderkerk aan de Amstel
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.set_resource_allocation(resources_with_ouderkerk)

In [ ]:
evaluator = Evaluator()
evaluator.add_metric("response_time", name="response time", description="Overall response time of all deployments.")
evaluator.add_metric("on_time", name="First TS on-time", vehicles="TS", first_only=True,
                     description="Overall on-time rate of first TS.", quantiles=None)
evaluator.add_metric("delay", name="Service area delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["OUDERKERK AAN DE AMSTEL"],
                     description="Response time delay of first TS for prio 1 incidents in affected service area.")

# run the experiment
experiment3b = ABTest(evaluator=evaluator)
experiment3b.set_scenario_A(sim_a, description="Ouderkerk aan de Amstel closed.")
experiment3b.set_scenario_B(sim_b, description="Ouderkerk aan de Amstel operating (1 TS with part time crew).")
experiment3b.run()

In [ ]:
experiment3b.print_results()

## Experiment 3.c: Adding a station in Bovenkerk
In this experiment, we add a station in between the two most southern stations in the region and station Amstelveen, since there is a relatively large area there without station. We place the station in demand location `13710002`.

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has the original station Ouderkerk aan de Amstel
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.set_resource_allocation(resources_with_ouderkerk)
sim_b.move_station("OUDERKERK AAN DE AMSTEL", "13710002", keep_name=False, new_name="BOVENKERK")

evaluator = Evaluator()
evaluator.add_metric("response_time", name="Response time", description="Overall response time of all deployments.")
evaluator.add_metric("on_time", name="First TS on-time", vehicles="TS", first_only=True,
                     description="Overall on-time rate of first TS.", quantiles=None)
evaluator.add_metric("delay", name="Amstelveen delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["AMSTELVEEN"],
                     description="Response time delay of first TS for prio 1 incidents in service area of station Amstelveen.")
evaluator.add_metric("delay", name="Uithoorn delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["UITHOORN"],
                     description="Response time delay of first TS for prio 1 incidents in service area of station Uithoorn.")
evaluator.add_metric("delay", name="Aalsmeer delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["AALSMEER"],
                     description="Response time delay of first TS for prio 1 incidents in service area of station Aalsmeer.")


experiment3c = ABTest(evaluator=evaluator)
experiment3c.set_scenario_A(sim_a, description="Baseline situation.")
experiment3c.set_scenario_B(sim_b, description="Additional station in Bovenkerk.")
experiment3c.run()
experiment3c.print_results()

# 4. Station status cycles.

## Experiment 4.a: Closing station Victor at night.
In this experiment, we close station Victor every night between 23PM and 7AM, as happened in real life.

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has Victor closed every night
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.set_daily_station_status("VICTOR", 23, 7, status="closed")

evaluator = Evaluator()
evaluator.add_metric("response_time", name="Response time", description="Overall response time of all deployments.")
evaluator.add_metric("on_time", name="First TS on-time", vehicles="TS", first_only=True,
                     description="Overall on-time rate of first TS.", quantiles=None)
evaluator.add_metric("response_time", name="SA response time", description="Response time of in service area Victor.",
                     locations=service_areas["VICTOR"])
evaluator.add_metric("delay", name="Victor delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["VICTOR"],
                     hours=[23, 0, 1, 2, 3, 4, 5, 6], description="Response time delay of first TS for prio 1 incidents in service area of station Victor during the closing hours.")

experiment4a = ABTest(evaluator=evaluator)
experiment4a.set_scenario_A(sim_a, description="Baseline situation.")
experiment4a.set_scenario_B(sim_b, description="Station Victor closed at night between 23 PM and 7 AM.")
experiment4a.run()
experiment4a.print_results()

## Experiment 4.b: Diemen operated by part time crew after office hours.
In this experiment, we close station Victor every night between 23PM and 7AM, as happened in real life.

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has Victor closed every night
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.set_daily_station_status("DIEMEN", 18, 8, status="parttime", days_of_week=[0, 1, 2, 3, 4])
sim_b.set_daily_station_status("DIEMEN", 8, 8, status="parttime", days_of_week=[5, 6])
print(sim_b.stations["DIEMEN"].status_table)

In [ ]:
evaluator = Evaluator()
evaluator.add_metric("response_time", name="Response time", description="Overall response time of all deployments.")
evaluator.add_metric("on_time", name="First TS on-time", vehicles="TS", first_only=True,
                     description="Overall on-time rate of first TS.", quantiles=None)
evaluator.add_metric("response_time", name="SA response time", description="Response time of in service area Diemen.",
                     locations=service_areas["DIEMEN"])
evaluator.add_metric("delay", name="Weekday delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["DIEMEN"],
                     hours=[18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7], days_of_week=[0, 1, 2, 3, 4],
                     description="Response time delay of first TS for prio 1 incidents in service area of station Diemen on weekdays after office hours.")

evaluator.add_metric("delay", name="Weekend delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["DIEMEN"], days_of_week=[5, 6],
                     description="Response time delay of first TS for prio 1 incidents in service area of station Diemen during the weekend.")

experiment4b = ABTest(evaluator=evaluator)
experiment4b.set_scenario_A(sim_a, description="Baseline situation.")
experiment4b.set_scenario_B(sim_b, description="Diemen operated by part time crews outside office hours.")
experiment4b.run()
experiment4b.print_results()

# 5. Backup protocols
In Amsterdam-Amstelland, some stations have both full time and part time personell. In such cases, a backup protocol can be used in which part time crews come to the fire station preventively when the full time crew is deployed in order to ensure a timely response for a possible second incident.

We test the usefulness of having such a protocol in place at the one fire station that has this in practice: Amstelveen.

## Experiment 5.a: Backup protocol at station Amstelveen.

In [ ]:
# scenario A is the baseline
sim_a = quick_load_simulator("baseline_simulator.pickle")
# scenario B has Victor closed every night
sim_b = quick_load_simulator("baseline_simulator.pickle")
sim_b.activate_backup_protocol("AMSTELVEEN")

evaluator = Evaluator()
evaluator.add_metric("response_time", name="Response time", description="Overall response time of all deployments.")
evaluator.add_metric("on_time", name="First TS on-time", vehicles="TS", first_only=True,
                     description="Overall on-time rate of first TS.", quantiles=None)
evaluator.add_metric("response_time", name="SA response time", description="Response time of in service area Amstelveen.",
                     locations=service_areas["AMSTELVEEN"])
evaluator.add_metric("delay", name="High risk delay", vehicles="TS", first_only=True, prios=1, locations=service_areas["AMSTELVEEN"],
                     description="Response time delay of first TS for prio 1 incidents in service area of station Amstelveen")

experiment4b = ABTest(evaluator=evaluator)
experiment4b.set_scenario_A(sim_a, description="Baseline situation.")
experiment4b.set_scenario_B(sim_b, description="Using a backup protocol at fire station Amstelveen.")
experiment4b.run()
experiment4b.print_results()

# 6. Increasing incident rates.